# Creation of H1 Dataset

The new HGG ISCCP dataset spans the years 1983-2009 and consists of a netCDF file for every 3 hour period. 
Trying to analyze PC-TAU information by accessing the raw data directly is inefficient -- therefore, the relevant
data from each file will be extracted and aggregated by year.

For now, the analyzed data will be stored as a numpy array (file ending as `.npy`) that can be accessed via the `np.load()` function.

Each `.npy` file will be a 3D numpy array with shape (12, 12, 64800) which corresponds to (time (month), weather state, gridbox).

# Description of Creation Process

The raw data is arranged in a HardDrive in directories of year, subdirectories of month, with raw netCDF files for every 3 hour period of that month.
The procedure to create an H1 dataset for a given year is as follows:

1. Extract the relevant variables from a netCDF file representing 3 hours of data
2. Normalize the gridbox data (pctaudist variable) using the n_total variable to convert from pixel count to percentage
3. Compute the Euclidean distance of each gridbox with every PC-TAU histogram from the accepted Weather States
4. Find the smallest distance for each gridbox to determine the corresponding Weather State of the gridbox
5. Increment the counter of the numpy matrix cell keeping track of the specific time, Weather State, and gridbox

For example, if the first gridbox of a given 3 hour period for the month of Janurary corresponds to Weather State 7, then the following happends: `data[0,7,gridbox] += 1`

### Note: Reference analysis.f90 for the functions used to normalize gridbox data and for computing gridbox weather states

A demonstration of creating a single year of analyzed data is below.

In [1]:
import os
import gc
import math
import time
import netCDF4
import analysispy
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
#%matplotlib

In [4]:
base_path = '/run/media/dtropf/Seagate Backup Plus Drive/hgg_data/'
save_path = '/home/dtropf/Documents/NASA/clouds/validation/test_data/'

# Load PC-TAU histograms of accepted Weather States
pctau_list = np.loadtxt(os.getcwd()+'/'+'pctau.txt').reshape(12, 7, 6).transpose((0,2,1))/100.0

# Years to make datasets (excludes final year via python syntax)
years = [str(i) for i in range(2010, 2013)]

files = []

for year in years:
    year_path = base_path+year
    all_data = np.zeros((12, 12, 360*180))
    t1 = time.time()
    
    # Loop into each month directory for the given year
    for i, f1 in enumerate(sorted(os.listdir(year_path))):
        #if 'ISCCP' in f1:
        month_path = year_path+'/'+f1

        # Loop over every 3 hour netCDF file
        for f2 in (sorted(os.listdir(month_path))):
            data_path = month_path+'/'+f2
            with netCDF4.Dataset(data_path) as data:
                pctaudist = data['n_pctaudist'][:,:,:]
                sqlonbeg = data['sqlon_beg'][:]
                sqlonend = data['sqlon_end'][:]
                eqlat = data['eqlat_index'][:]
                n_total = data['n_total'][:]

                # Normalize pctaudist from pixel count to 
                # percentage using fortran function
                pctaudist_norm = analysispy.fnorm(pctaudist, n_total, pctaudist.mask)

                # Calculate weatherstate for each of the 64800 gridboxes
                ws_num = analysispy.findws(pctaudist_norm, pctau_list, pctaudist.mask, sqlonbeg, sqlonend, eqlat)

                # Reshape data to fit the shape of (12, 12, 64800)
                for j in range(12):
                    all_data[i,j,:] = all_data[i,j,:] + ws_num[:,:,j].reshape(360*180)

                # Memory cleanup
                del data
                del pctaudist
                del sqlonbeg
                del eqlat
                del n_total
                gc.collect()
    t2 = time.time()
            
    print('Time for ',year,t2-t1)
    
    # Save 3D numpy array
    np.save(save_path+'H1_'+year, all_data)
    
    # Memory cleanup
    del all_data
    gc.collect()

Time for  2010 506.1533224582672
Time for  2011 512.2897651195526
Time for  2012 522.095691204071


# Using H1 Dataset

Analyzed data is stored on the level of month of a given year. This means that the max temporal resolution of the data is month.

For example, should a map of Weather State 6 of July of 1999 be desired, then one can acquire that specific data like so:

>> `data_path = 'H1_1999.npy'`

>>`data = np.load(data_path)`

>>`desired_data = data[7,6,:].reshape(360, 180)`

Note that all data along the 3rd axis is arranged in Equal-Angle format and can be reshaped in the format of LON,LAT at will (another rotation of data is necessary for LAT, LON arrangment).

See the following for examples of H1 Dataset use:

1. h1_wsmaps.ipynb for vizualizing the dataset
2. api_test.ipynb for easily accessing the data using custom functions for such
3. 